In [ ]:
#1. IMPORTING PACKAGES AND LIBRARIES
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext jupyternotify

import os
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import GoogleTiles as moa
from cycler import cycler
import scipy

import glob
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import xarray as xr
import pyproj
import osr
import datetime
import subprocess
import shutil
import numpy as np
from ipywidgets import interact
import pandas as pd
import pickle
from scipy.signal import savgol_filter
import time


# import our own small lib. I access this one through PyCharm
import nsidc

In [11]:
%%notify
import time
time.sleep(10)
print('Finished!')

Finished!


<IPython.core.display.Javascript object>

In [14]:

freq = '37' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
res = '25'  # can also be 3.125,  12.5 or 25, although this depends on the chosen frequency as well
# freq = '36' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
HV = 'V' # Horizontal or Vertical polarisation
AD = 'A' # Ascending or descending, descending contains imagery during the day, probably showing more contrast
bounds = [( -19., 20.),
          (-11., 27.),
         ]

# start_date = datetime.datetime(2002, 5, 29) # Start of the AMSR-E period
# start_date = datetime.datetime(1988, 1, 1) # Start of the SSMI F08
# end_date = datetime.datetime(1991, 12, 31)  # end of the SSMI F08
# start_date = datetime.datetime(1992, 1, 1) # Start of the SSMI F10
# end_date = datetime.datetime(1995, 12, 31)  # end of the SSMI F10
# start_date = datetime.datetime(1996, 1, 1) # Start of the SSMI F13
# end_date = datetime.datetime(2000, 12, 31)  # end of the SSMI F13
start_date = datetime.datetime(2001, 1, 1) # Start of the SSMI F15
end_date = datetime.datetime(2017, 6, 30)  # end of the SSMI F15


# start_date = datetime.datetime(2004, 1, 1) # date after data recovery
# end_date = datetime.datetime(2009, 12, 31)  # only a few days to make a demonstration
# end_date = datetime.datetime(2011, 5, 28)  # use this end date to cover the whole AMSR-E period.
# end_date = datetime.datetime(2002, 1, 2)

# url = nsidc.make_measures_url(start_date, res, freq, HV, AD) #Uncheck this line to obtain the download url. This can be used if th combination of parameters is not coresponding to a download file

try:
    credential_fn = os.path.abspath('../../nsidc_credentials.txt')
    username, password = str(np.loadtxt(credential_fn, dtype=np.str)).split(',')
except:
    print('No credential file found, please put a txt file with <user>,<pass> in {:s}'.format(credential_fn))
    
username = 'okeunen'
password = 'Orkeunen1'

url, success = nsidc.download_measures(freq, res, HV, AD, start_date, username, password) #To check url and success of download -> 0 = succes
fn_out_prefix = 'NSIDC_Test'

NotImplementedError 

# download command
nsidc.download_measures_ts(freq, res, HV, AD, start_date, end_date, bounds, fn_out_prefix, username, password)
url 


print('----------------------------')
if success==0:
    print('SUCCESSFULL DOWNLOAD')
    print('----------------------------')
    print('SETTINGS:'' Frequency:',freq,'GHz.', 
      ' Resolution:',res, 'km.',
      ' Polarization:', HV,'.' 
      ' Ascending/descending:', AD)

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.01/NSIDC-0630-EASE2_T25km-F15_SSMI-2001001-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.02/NSIDC-0630-EASE2_T25km-F15_SSMI-2001002-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.03/NSIDC-0630-EASE2_T25km-F15_SSMI-2001003-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.04/NSIDC-0630-EASE2_T25km-F15_SSMI-2001004-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.05/NSIDC-0630-EASE2_T25km-F15_SSMI-2001005-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.06/NSIDC-0630-EASE2_T25km-F15_SSMI-2001006-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.01.07/NSIDC-0630-EASE2_T25km-F15_SSMI-2001007-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.02/NSIDC-0630-EASE2_T25km-F15_SSMI-2001061-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.03/NSIDC-0630-EASE2_T25km-F15_SSMI-2001062-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.04/NSIDC-0630-EASE2_T25km-F15_SSMI-2001063-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.05/NSIDC-0630-EASE2_T25km-F15_SSMI-2001064-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.06/NSIDC-0630-EASE2_T25km-F15_SSMI-2001065-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.07/NSIDC-0630-EASE2_T25km-F15_SSMI-2001066-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.03.08/NSIDC-0630-EASE2_T25km-F15_SSMI-2001067-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.01/NSIDC-0630-EASE2_T25km-F15_SSMI-2001121-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.02/NSIDC-0630-EASE2_T25km-F15_SSMI-2001122-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.03/NSIDC-0630-EASE2_T25km-F15_SSMI-2001123-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.04/NSIDC-0630-EASE2_T25km-F15_SSMI-2001124-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.05/NSIDC-0630-EASE2_T25km-F15_SSMI-2001125-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.06/NSIDC-0630-EASE2_T25km-F15_SSMI-2001126-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.05.07/NSIDC-0630-EASE2_T25km-F15_SSMI-2001127-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.06.30/NSIDC-0630-EASE2_T25km-F15_SSMI-2001181-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.07.01/NSIDC-0630-EASE2_T25km-F15_SSMI-2001182-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.07.02/NSIDC-0630-EASE2_T25km-F15_SSMI-2001183-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.07.03/NSIDC-0630-EASE2_T25km-F15_SSMI-2001184-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.07.04/NSIDC-0630-EASE2_T25km-F15_SSMI-2001185-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.07.05/NSIDC-0630-EASE2_T25km-F15_SSMI-2001186-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.07.06/NSIDC-0630-EASE2_T25km-F15_SSMI-2001187-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.08.29/NSIDC-0630-EASE2_T25km-F15_SSMI-2001241-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.08.30/NSIDC-0630-EASE2_T25km-F15_SSMI-2001242-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.08.31/NSIDC-0630-EASE2_T25km-F15_SSMI-2001243-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.09.01/NSIDC-0630-EASE2_T25km-F15_SSMI-2001244-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.09.02/NSIDC-0630-EASE2_T25km-F15_SSMI-2001245-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.09.03/NSIDC-0630-EASE2_T25km-F15_SSMI-2001246-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2001.09.04/NSIDC-0630-EASE2_T25km-F15_SSMI-2001247-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

...

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.24/NSIDC-0630-EASE2_T25km-F15_SSMI-2016268-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.25/NSIDC-0630-EASE2_T25km-F15_SSMI-2016269-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.26/NSIDC-0630-EASE2_T25km-F15_SSMI-2016270-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.27/NSIDC-0630-EASE2_T25km-F15_SSMI-2016271-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.28/NSIDC-0630-EASE2_T25km-F15_SSMI-2016272-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.29/NSIDC-0630-EASE2_T25km-F15_SSMI-2016273-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.09.30/NSIDC-0630-EASE2_T25km-F15_SSMI-2016274-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.23/NSIDC-0630-EASE2_T25km-F15_SSMI-2016328-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.24/NSIDC-0630-EASE2_T25km-F15_SSMI-2016329-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.25/NSIDC-0630-EASE2_T25km-F15_SSMI-2016330-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.26/NSIDC-0630-EASE2_T25km-F15_SSMI-2016331-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.27/NSIDC-0630-EASE2_T25km-F15_SSMI-2016332-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.28/NSIDC-0630-EASE2_T25km-F15_SSMI-2016333-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2016.11.29/NSIDC-0630-EASE2_T25km-F15_SSMI-2016334-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.21/NSIDC-0630-EASE2_T25km-F15_SSMI-2017021-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.22/NSIDC-0630-EASE2_T25km-F15_SSMI-2017022-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.23/NSIDC-0630-EASE2_T25km-F15_SSMI-2017023-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.24/NSIDC-0630-EASE2_T25km-F15_SSMI-2017024-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.25/NSIDC-0630-EASE2_T25km-F15_SSMI-2017025-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.26/NSIDC-0630-EASE2_T25km-F15_SSMI-2017026-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.01.27/NSIDC-0630-EASE2_T25km-F15_SSMI-2017027-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.22/NSIDC-0630-EASE2_T25km-F15_SSMI-2017081-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.23/NSIDC-0630-EASE2_T25km-F15_SSMI-2017082-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.24/NSIDC-0630-EASE2_T25km-F15_SSMI-2017083-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.25/NSIDC-0630-EASE2_T25km-F15_SSMI-2017084-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.26/NSIDC-0630-EASE2_T25km-F15_SSMI-2017085-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.27/NSIDC-0630-EASE2_T25km-F15_SSMI-2017086-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.03.28/NSIDC-0630-EASE2_T25km-F15_SSMI-2017087-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.21/NSIDC-0630-EASE2_T25km-F15_SSMI-2017141-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.22/NSIDC-0630-EASE2_T25km-F15_SSMI-2017142-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.23/NSIDC-0630-EASE2_T25km-F15_SSMI-2017143-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.24/NSIDC-0630-EASE2_T25km-F15_SSMI-2017144-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.25/NSIDC-0630-EASE2_T25km-F15_SSMI-2017145-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.26/NSIDC-0630-EASE2_T25km-F15_SSMI-2017146-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2017.05.27/NSIDC-0630-EASE2_T25km-F15_SSMI-2017147-37V-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

In [ ]:
# ds_36ghz_25km_V_A_2002 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2002.nc')
# ds_36ghz_25km_V_A_2003 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2003.nc')
# ds_36ghz_25km_V_A_2004 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2004.nc')
# ds_36ghz_25km_V_A_2005 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2005.nc')
# ds_36ghz_25km_V_A_2006 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2006.nc')
# ds_36ghz_25km_V_A_2007 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2007.nc')
# ds_36ghz_25km_V_A_2008 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2008.nc')
# ds_36ghz_25km_V_A_2009 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_2009.nc')
# ds_36ghz_25km_V_A_2010 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_20010.nc')
# ds_36ghz_25km_V_A_2011 = xr.open_dataset('/Users/oscarkeunen/Documents/1. TU Delft/1. Msc - Watermanagement/Afstuderen/satellite-cookbook-master/NSIDC-AMSRE/Sat_Data/NSIDC_Test_20011.nc')

# ds_36ghz_3km_V_A_2002 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2003 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2004 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2005 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2006 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2007 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2008 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2009 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2010 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')
# ds_36ghz_3km_V_A_2011 = xr.open_dataset('C:\\Users\\908138\\githubdata\\37ghz_25km_H_A\\NSIDC-0630-EASE2_T25km-AQUA-AMSRE-36H-A_2010.nc')

In [10]:
download_string

'wget --http-user=okeunen --http-password=Orkeunen1 --load-cookies mycookies.txt --save-cookies mycookies.txt --keep-session-cookies --no-check-certificate --auth-no-challenge -r --reject "index.html*" -np -e robots=off https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2002.05.29/NSIDC-0630-EASE2_T25km-AQUA_AMSRE-2002149-36V-A-GRD-RSS-v1.3.nc'